# (1)

In [1]:
import os
import pandas as pd
import urllib.request as re
%matplotlib inline
pd.set_option('display.max_rows', 10)

In [2]:
if not os.path.isfile("./Online_Retail.csv"):
    print('start download Online_Retail.xlsx')
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
    req.urlretrieve(url, "Online_Retail.xlsx")
    trans = pd.read_excel("Online_Retail.xlsx", sheet_name="Online Retail")
    trans.to_csv("./Online_Retail.csv")
else:
    print('Online_Retail.csv exists')

Online_Retail.csv exists


In [3]:
trans = pd.read_csv("./Online_Retail.csv")

In [4]:
trans

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...,...
541904,541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [5]:
display('StockCodeのカウント',trans.StockCode.value_counts())
display('欠損値の全部を合計', trans.isnull().values.sum()) # Trueの数をカウント
display('欠損値の有る/無し', trans.isnull().any()) # 欠損値が無い: False, 有る: True
display('欠損値の列毎の集計', trans.isnull().sum())
#display('欠損値の各列の集計', trans.isnull().values) # True/Falseの表示
# CusomerIDに欠損値を持つレコードを削除

trans_dropna = trans.dropna(subset = ['CustomerID'])

# print('CustomerID has null?: ',trans.CustomerID.isnull().any())
# trans_dropna = trans.CustomerID.dropna()
# print('trans_dropna', type(trans_dropna))
# trans_dropna.isnull().sum()

# trans_dropna
# trans_dropna.StockCode.value_counts().head(5)

'StockCodeのカウント'

85123A      2313
22423       2203
85099B      2159
47566       1727
20725       1639
            ... 
DCGS0071       1
47563B         1
23605          1
85179B         1
72789          1
Name: StockCode, Length: 4070, dtype: int64

'欠損値の全部を合計'

136534

'欠損値の有る/無し'

Unnamed: 0     False
InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
dtype: bool

'欠損値の列毎の集計'

Unnamed: 0          0
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
##### 前処理 ######
def getFirstCaractor(x):
    return str(x)[0]
trans['cancel_flg'] = trans.InvoiceNo.map(getFirstCaractor)
print(trans.groupby('cancel_flg').size())
trans = trans[(trans.cancel_flg == '5') & (trans.CustomerID.notnull())]
################

cancel_flg
5    532618
A         3
C      9288
dtype: int64


In [7]:
# データフレームからStockCodeの数をカウントして上位5件を表示
trans.StockCode.value_counts().head(5)

85123A    2035
22423     1724
85099B    1618
84879     1408
47566     1397
Name: StockCode, dtype: int64

# (2)

## すべての購買データのバスケット(InvoiceNo)数をカウントする

In [8]:
trans_ALL = set(trans.InvoiceNo)
print("全ての購買データ      :", end='')
print(len(trans_ALL))

全ての購買データ      :18536


## 商品85123Aを購入したバスケット数をカウントする

In [9]:
#for i in trans_ALL:
s_b_85123A = trans.StockCode == "85123A"
s_trans_85123A = trans[s_b_85123A].InvoiceNo
display(len(s_trans_85123A))
# invoiceNoが重複していることが確認できる。一つのInvoceNoで複数回85123Aを購入している。
display('InvoiceNo value_counts()',trans[s_b_85123A].InvoiceNo.value_counts())
# 重複を回避してinvoiceNoをカウントする
print('85123Aを購入:   ', end='')
print(len(set(trans[s_b_85123A].InvoiceNo)))


2035

'InvoiceNo value_counts()'

568301    3
547670    3
551057    3
571461    3
560271    3
         ..
543166    1
549837    1
580062    1
548978    1
565439    1
Name: InvoiceNo, Length: 1978, dtype: int64

85123Aを購入:   1978


## 商品47566を購入したバスケット数をカウントする

In [10]:
s_b_47566 = trans.StockCode == '47566'
s_trans_47566 = trans[s_b_47566].InvoiceNo
print('47566を購入: ', len(set(s_trans_47566)))

47566を購入:  1380


## 商品85123Aかつ商品47566を購入したバケット数をカウントする

In [11]:
s_b_85123A = trans.StockCode == "85123A"
s_b_47566 = trans.StockCode == '47566'
# set(trans[s_b_47566].InvoiceNo)　 & set(trans[s_b_85123A].InvoiceNo)　

s1 = set(trans[s_b_47566].InvoiceNo)
s2 = set(trans[s_b_85123A].InvoiceNo)

print('85123A かつ 47566を購入: ', len(s1 & s2))

85123A かつ 47566を購入:  279


# 3

## 3-1 商品85123Aを購入した人の中で、商品47566を購入した人の割合を示す信頼度（C)を計算する

In [31]:
c_num_85123A = len(set(trans[s_b_85123A].InvoiceNo))
c_num_47566 = len(set(s_trans_47566))
c_num_85123A_and_47566 = len(s1 & s2)
confidence = c_num_85123A_and_47566/c_num_85123A
print('信頼度: {:.3f}'.format(confidence))

信頼度: 0.141


## 3-2 承認85123Aと商品47566が同時に購入されたかどうかを示す支持度(S_XY_)を計算する

In [36]:
c_num_all = len(trans_ALL)
support = c_num_85123A_and_47566/c_num_all
print('支持度: {:.3f}'.format(support))

支持度: 0.015


## 3-3 商品85123Aを購入するなら商品47566も購入するというルールのリフト値を求めよう。

In [39]:
support_47566 = c_num_47566/c_num_all
lift = confidence/support_47566
print('85123Aを購入するなら商品47566も購入するリフト値: {:.3f}'.format(lift))
# 1.0をこえているので、47566を単体で売るより、85123Aを購入した人におすすめしたほうが効果がある。

85123Aを購入するなら商品47566も購入するリフト値: 1.895


# Series map

In [13]:
df_test = pd.DataFrame({"a": ['first','second','third','fourth','fifth']})

def testFunc(x):
    print ('In testFunc. arg: ', x)
    return x + "_extra"

a_ = df_test.a.map(testFunc)
print('returned type: ', type(a_))
display('returned value: ', a_)

In testFunc. arg:  first
In testFunc. arg:  second
In testFunc. arg:  third
In testFunc. arg:  fourth
In testFunc. arg:  fifth
returned type:  <class 'pandas.core.series.Series'>


'returned value: '

0     first_extra
1    second_extra
2     third_extra
3    fourth_extra
4     fifth_extra
Name: a, dtype: object

In [14]:
a_ = df_test.a.map(lambda x: str(x)[0])
print(type(a_))
print(a_)

<class 'pandas.core.series.Series'>
0    f
1    s
2    t
3    f
4    f
Name: a, dtype: object


# Set

In [15]:
# set型オブジェクト. 重複を回避
s = {1,2,2,3,1,4}
print(s)
print('type: ', type(s))

{1, 2, 3, 4}
type:  <class 'set'>


In [16]:
# 順序は保証しない
s = {1.23, '百',(0,1,2), '百'}
print(s)

{(0, 1, 2), 1.23, '百'}


In [17]:
# 100や100.0も重複
s = {100, 100.00}
s

{100}

In [18]:
# 空のset型はdict型と解釈される
s = {}
print(type(s))
# 空のset型はコンストラクタで生成
s = set()
print(type(s))

<class 'dict'>
<class 'set'>


In [19]:
# コンストラクタで生成その２
l = [1,2,2,3,1,4]
s = set(l)
print(type(s))
s

<class 'set'>


{1, 2, 3, 4}

In [20]:
# 集合内包表記は{}で囲む
s = {i**2 for i in range(5)}
print(type(s))
print(s)

<class 'set'>
{0, 1, 4, 9, 16}


In [21]:
# 要素数
len(s)

5

In [22]:
# 要素を追加
s.add(25)
s

{0, 1, 4, 9, 16, 25}

In [23]:
# 要素を削除
s.discard(1)
s.discard(-10)
print(s)

#s.remove(-10) # KeyErrorが発生

v = s.pop()
print("v: ", v)
print("s: ", s)

s.clear()
print("s: ", s)

{0, 4, 9, 16, 25}
v:  0
s:  {4, 9, 16, 25}
s:  set()


# Dropna subset

In [24]:
import numpy as np
df_test = pd.DataFrame({ "a": [1,2,3,4,5], "b": [6,7,np.nan,9,10],"c":[11,12,13,14,15] })
display('data frame',df_test)
display('not null', df_test.notnull())
df_test.dropna(subset=["b"])

'data frame'

,a,b,c
0,1,6.0,11
1,2,7.0,12
2,3,NaN,13
3,4,9.0,14
4,5,10.0,15


'not null'

,a,b,c
0,True,True,True
1,True,True,True
2,True,False,True
3,True,True,True
4,True,True,True


,a,b,c
0,1,6.0,11
1,2,7.0,12
3,4,9.0,14
4,5,10.0,15


# groupby

In [25]:
df_test = pd.DataFrame({ "a": [1,2,2,2,5], "b": [6,7,np.nan,9,10],"c":[11,12,13,14,15] })
display(df_test)
for i, j in df_test.groupby('a'):
    print(i)
    print(j)
    print('---')

,a,b,c
0,1,6.0,11
1,2,7.0,12
2,2,NaN,13
3,2,9.0,14
4,5,10.0,15


1
   a    b   c
0  1  6.0  11
---
2
   a    b   c
1  2  7.0  12
2  2  NaN  13
3  2  9.0  14
---
5
   a     b   c
4  5  10.0  15
---


# dataframe

In [26]:
df_test = pd.DataFrame({ "a": [1,2,2,2,5], "b": [6,7,np.nan,9,10],"c":[11,12,13,14,15] })
display('original', df_test)
s_a_2 =(df_test["a"]==2)
display('filter descripton with a == 2',s_a_2)
display('filter with a== 2', df_test[s_a_2])
display('type of s_a_2', type(s_a_2))

s_a = pd.Series([False, False, True, True, True])
display(df_test[s_a])

'original'

,a,b,c
0,1,6.0,11
1,2,7.0,12
2,2,NaN,13
3,2,9.0,14
4,5,10.0,15


'filter descripton with a == 2'

0    False
1     True
2     True
3     True
4    False
Name: a, dtype: bool

'filter with a== 2'

,a,b,c
1,2,7.0,12
2,2,NaN,13
3,2,9.0,14


'type of s_a_2'

pandas.core.series.Series

,a,b,c
2,2,NaN,13
3,2,9.0,14
4,5,10.0,15
